# Airline Network Revenue Management

In this notebook, I will implement linear programming approach for a simple airline network revenue management. The example is taken from the book [Pricing and Revenue Optimization](https://www.sup.org/books/title/?id=31628) - Second Edition, pages 248-253. This example is in the chapter 10 of the book where the author Robert Phillips discusses network revenue management.

---

## Example
Consider a simple [hub-and-spoke](https://en.wikipedia.org/wiki/Spoke%E2%80%93hub_distribution_paradigm) airline network where an airline offers two flights:

San Francisco (SFO) ----------> Denver (DEN) ----------------> St. Louis (STL)

*Flight 1*: SFO ---> DEN

*Flight 2*: DEV ---> STL

Given these two flights, the airline can create three products:
- San Francisco to Denver flight
- Denver to St. Louis flight
- San Francisco to St. Louis flight

Essentially an airline product is simply an origin-destination combination. For each product, the airline can offer different fare classes, like full fare, discount fare, and deep discount fare, where in general the price is getting lower and lower, respectively. For this example, we assume that this airline offers full fare and discount fare. Accoridngly, the airline can sell six types of flight tickets to the customers:
1. SFO ---> DEV full fare
2. SFO ---> DEV discount fare
3. DEV ---> STL full fare
4. DEV ---> STL discount fare
5. SFO ---> STL full fare
6. SFO ---> STL discount fare

This combination of product and fare class is called *origin-destination fare class*, or ODF in traditional revenue management. Several ket facts about those ODFs:
- Each ODF is a combination of product and fare class.
- Each ODF has an associated demand. Demands are observed in the ODF level.
- Airline fares are priced at ODF level.
- Each ODF consumes one or multiple resources. For instance, SFO ---> DEV full fare consumes one seat from SFO ---> DEV flight; SFO ---> STL discount fare consumes one flight seat from each of the SFO ---> DEV flight and DEV --> STL flight.

---

The way airline bookings works is that consumers make advance purchase requests on an upcoming flight and airlines make decisions on accepting or rejecting those advance purchases. The tradeoff airlies face is that if they accept a request with discount fare, it might dislace a potential full fare request in the future. Also, if a customer wants to book a fligt that spans multiple legs, it is essentially consuming multiple legs. For airlines the revenue from such booking is lower than the sum of the fare they can collect by selling each resource individually to local booking requests. For instance, airlines might accept a SFo ---> STL booking request for 300.00 dollars where they could have sold two separate single-leg local flights for a total of 350 dollars, 200.00 for the first flight and 150.00 for the second flight. This means lost revenue for the airline.

---

That's why we need a systematic way of evaluating our decision, and that is where linear programming comes in. Before we start, let's provide some basic information about each ODFs of the airline:

| Number | ODF                     | Fare (\$) | Demand |
|--------|-------------------------|-----------|--------|
| 1      |SFO -> DEV full fare     | 150       | 30     |
| 2      |SFO -> DEV discount fare | 100       | 60     |
| 3      |DEV -> STL full fare     | 120       | 20     |
| 4      |DEV -> STL discount fare | 80        | 80     |
| 5      |SFO -> STL full fare     | 250       | 30     |
| 6      |SFO -> STL discount fare | 170       | 40     |

SFO -> DEV flight has a capacity of 100 seats and DEV -> STL flight has a capacity of 120 seats. 

---

Let's define some general notation to this type of problem, then we come back to our problem. 

We first have resources and ODFs. Let's assume we have $m$ resouces (in our case $m=2$) and $n$ ODFs (in our case $n=6$) with $n \geq m$. We can use subscript $i$ to index resources and the subscript $j$ to index ODF. Each resource $i$ has a limited capacity of $C_i>0$. Each ODF has a *known* demand $d_j>0$ and a net contribution margin of $p_j>0$. 

Let $x_j\geq0$ be the amount of ODF $j$ we will allow to book. Then our problem is to find the values of $x_j$ for $j=1, 2, ..., n$ that maximizes total net contriution subjest to the constrained available capacity.

We need one more variable to address the resource consumption instance of each ODF. To do this, we define the incidence variable $a_{ij}$ as follows:
$$
\begin{equation}
a_{ij} = \begin{cases}
1 \qquad & \text{ if resource } i \text{ is used in ODF }j\\
0 \qquad &\text{otherwise}
\end{cases}
\end{equation}
$$

---
Put the incidence variable in the context of our example, it looks like this:

|              |   |   | ODF ($j$)|   |   |   |
|--------------|---|---|----------|---|---|---|
|Resource ($i$)| 1 | 2 | 3        | 4 | 5 | 6 |
|--------------|---|---|----------|---|---|---|
|1             | 1 | 1 | 0        | 0 | 1 | 1 |
|2             | 0 | 0 | 1        | 1 | 1 | 1 |

For instance, both SFO -> DEV full and discount fare consumes a flight seat on SFO -> DEV flight.

Once we have all these, we are now ready to write our linear programming formulation:

$$
\begin{align}
&\max_{x_j}\,\sum_{j=1}^{n}p_jx_j,\\
\text{subject to} &\\
&\sum_{j=1}^{n}a_{ij}x_j \leq C_i\qquad \text{ for all }i,\\
& x_j \leq d_j, \qquad \text{ for all }j,\\
& x_j \geq 0 , \qquad \text{ for all }j.\\
\end{align}
$$

Next, we go back to our example and try to write the airline's problem in a more understandable way. After all, not everyone likes this *complex* notation of LP.

$$
\begin{align}
\text{maximize } \qquad & 150x_1 + 100x_2 + 120x_3 + 80x_4 + 250x_5 + 170x_6\\
\text{subject to } \qquad &\\
& x_1 + x_2 + x_5 + x_6 \leq 100,\\
& x_3 + x_4 + x_5 +x_6 \leq 120,\\
& x_1 \leq 30,\\
& x_2 \leq 60,\\
& x_3 \leq 20,\\
& x_4 \leq 80,\\
& x_5 \leq 30,\\
& x_6 \leq 40,\\
& x_1, x_2, x_3, x_4, x_5, x_6 \geq 0.
\end{align}
$$

Looking back our simple LP example, we can write it as a matrix form so that its python implementation will be very clear. The matrix form is for constraints $Gx\leq h$.

$$
\begin{bmatrix}
1 & 1 & 0 & 0 & 1 & 1\\
0 & 0 & 1 & 1 & 1 & 1\\
1 & 0 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 1 & 0 & 0 & 0\\
0 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x_1\\
x_2\\
x_3\\
x_4\\
x_5\\
x_6
\end{bmatrix}
\leq
\begin{bmatrix}
100\\
120\\
30\\
60\\
20\\
80\\
30\\
40
\end{bmatrix}
$$

## Python Implementation

In [1]:
import numpy as np
from cvxopt import matrix, solvers

In [14]:
c = matrix([-150., -100., -120., -80., -250., -170.])
G = matrix([[1., 0., 1., 0., 0., 0., 0., 0., -1., 0., 0., 0., 0., 0.], 
            [1., 0., 0., 1., 0., 0., 0., 0., 0., -1., 0., 0., 0., 0.], 
            [0., 1., 0., 0., 1., 0., 0., 0., 0., 0., -1., 0., 0., 0.],
            [0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., -1., 0., 0.],
            [1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., -1., 0.],
            [1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., -1.]])
h = matrix([100., 120., 30., 60., 20., 80., 30., 40., 0., 0., 0., 0., 0., 0.])

In [15]:
sol = solvers.lp(c, G, h)
print(sol['x'])
print(-sol['primal objective'])

     pcost       dcost       gap    pres   dres   k/t
 0: -2.1141e+04 -4.2921e+04  1e+04  0e+00  4e-01  1e+00
 1: -2.3166e+04 -2.6553e+04  2e+03  4e-16  6e-02  2e+01
 2: -2.3784e+04 -2.4468e+04  4e+02  2e-16  1e-02  6e+00
 3: -2.3975e+04 -2.4106e+04  7e+01  3e-16  3e-03  1e+00
 4: -2.3999e+04 -2.4002e+04  1e+00  1e-16  4e-05  5e-02
 5: -2.4000e+04 -2.4000e+04  1e-02  3e-16  4e-07  5e-04
 6: -2.4000e+04 -2.4000e+04  1e-04  1e-16  4e-09  5e-06
Optimal solution found.
[ 3.00e+01]
[ 4.00e+01]
[ 2.00e+01]
[ 7.00e+01]
[ 3.00e+01]
[ 8.85e-07]

23999.999948364362


The optimal allocation decision is 30, 40, 20, 70, 30, and 0. With this allocation the airline can achieve revenue of \$24,000.